# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Необходимо построить модель с предельно большим значением *F1*-меры.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

Импортируем все необходимые инструменты

In [4]:
!pip install scikit-learn==1.1.3
!pip install imblearn

In [1]:
import pandas as pd
from sklearn.model_selection import (train_test_split, GridSearchCV)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    f1_score, roc_auc_score, accuracy_score, recall_score, precision_score)
from sklearn.preprocessing import (StandardScaler, OneHotEncoder)
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import warnings
# настройки
warnings.filterwarnings("ignore")

In [2]:
def get_data_info(data):
    display(data.sample(5))
    display(data.info())
    display(data.describe(include='all'))

In [3]:
try:
    df = pd.read_csv('Churn.csv') #загрузим данные
except:
    df = pd.read_csv('/datasets/Churn.csv') #загрузим данные

Изучим данные

In [4]:
get_data_info(df)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
5485,5486,15753837,Young,573,Spain,Male,38,4.0,0.00,2,1,1,196517.43,0
3713,3714,15657937,Lord,709,Germany,Male,22,0.0,112949.71,1,0,0,155231.55,0
8090,8091,15601324,Black,697,France,Female,48,1.0,0.00,2,1,1,87400.53,0
163,164,15680772,Hu,721,Spain,Female,36,2.0,0.00,2,1,1,106977.80,0
1393,1394,15646276,Metcalfe,831,France,Female,32,2.0,146033.62,1,1,0,191260.74,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


None

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000,10000.000000,10000,10000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
unique,NaN,NaN,2932,NaN,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,Smith,NaN,France,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,32,NaN,5014,5457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,5000.50000,1.569094e+07,NaN,650.528800,NaN,NaN,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,NaN,96.653299,NaN,NaN,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,NaN,350.000000,NaN,NaN,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,NaN,584.000000,NaN,NaN,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,NaN,652.000000,NaN,NaN,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,NaN,718.000000,NaN,NaN,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000


Целевой признак в столбце 'Exited'

Столбцы "RowNumber", 'CustomerId' и "Surname" не помогут в предсказывании целевого признака, к тому же значение "RowNumber" можно легко восстановить по индексу. Уберем эти столбцы из данных.

In [5]:
df_prepared =  df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [8]:
df_prepared.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [9]:
df_prepared['Tenure'].isna().sum()

909

Пропуски есть только в столбце 'Tenure'. Все типы данных правильные. Возможно, данные о продолжительности сотрудничества отсутствуют для действующих клиетов.

In [10]:
df_prepared[df_prepared['Tenure'].isna()]['Exited']

30      1
48      0
51      0
53      1
60      0
       ..
9944    0
9956    1
9964    0
9985    0
9999    0
Name: Exited, Length: 909, dtype: int64

Предположение оказалость неверным. Удалим данные с пропусками, т.к. наличие пропусков помещает обучению модели.

In [11]:
df_prepared = df_prepared[df_prepared['Tenure'].notna()]

Разделим данные на features и target и разобьем данные на обучающую и валидационную выборки

In [12]:
features = df_prepared.drop('Exited', axis=1)

In [13]:
target = df_prepared['Exited']

Разделим данные на обучающую, тестовую и валидационную выборки

In [14]:
features_train, features_tmp, target_train, target_tmp = train_test_split(features, target, test_size=0.4, random_state=12345) #0.6 данных уходят в обучающую выборку, еще 0.4 
#мы дальше поделим на тестовую и валидационную по 0.2 данных в каждой

In [15]:
features_test, features_valid, target_test, target_valid = train_test_split(
    features_tmp, target_tmp, test_size=0.5, random_state=12345) #Получаем тестовую и валидационную выборки

In [16]:
features_train.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
9344,727,France,Female,28,2.0,110997.76,1,1,0,101433.76
3796,537,France,Male,26,7.0,106397.75,1,0,0,103563.23
7462,610,France,Male,40,9.0,0.00,1,1,1,149602.54
1508,576,France,Male,36,6.0,0.00,2,1,1,48314.00
4478,549,France,Male,31,4.0,0.00,2,0,1,25684.85


In [17]:
#категориальные признаки для OHE 
ohe_features = features_train.select_dtypes(include='object').columns.to_list()
ohe_features

['Geography', 'Gender']

In [18]:
#численные признаки
#'HasCrCard', 'IsActiveMember' — категориальные бинарный признак.
num_features = features_train.select_dtypes(exclude='object').columns.to_list()
num_features.remove('HasCrCard')
num_features.remove('IsActiveMember')
num_features

['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [19]:
# drop='first' удаляет первый признак из закодированных:
# таким образом обходим dummy-ловушку
# задаём handle_unknown='ignore':
# игнорируется ранее невстречающиеся значения признака (при transform)
encoder_ohe = OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False)

In [20]:
# обучаем энкодер на заданных категориальных признаках тренировочной выборки
encoder_ohe.fit(features_train[ohe_features])

OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False)

In [21]:
# добавляем закодированные признаки в features_train
# encoder_ohe.get_feature_names_out() позволяет получить названия колонок
features_train[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(features_train[ohe_features])

In [22]:
# удаляем незакодированные категориальные признаки (изначальные колонки)
features_train = features_train.drop(ohe_features, axis=1)

In [23]:
# создаём скелер
scaler = StandardScaler()

In [24]:
# обучаем его на численных признаках тренировочной выборки, трансформируем её же
features_train[num_features] = scaler.fit_transform(features_train[num_features])

In [25]:
# смотрим на результат
features_train.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
9344,0.809075,-1.039327,-1.025995,0.554904,-0.908179,1,0,0.019508,0.0,0.0,0.0
3796,-1.152518,-1.227561,0.696524,0.480609,-0.908179,0,0,0.056167,0.0,0.0,1.0
7462,-0.398853,0.090079,1.385532,-1.237830,-0.908179,1,1,0.848738,0.0,0.0,1.0
1508,-0.749875,-0.286389,0.352020,-1.237830,0.809300,1,1,-0.894953,0.0,0.0,1.0
4478,-1.028628,-0.756975,-0.336987,-1.237830,0.809300,0,1,-1.284516,0.0,0.0,1.0


In [26]:
# энкодером, который обучен на ТРЕНИРОВОЧНОЙ ВЫБОРКЕ, кодируем тестовую
features_test[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(features_test[ohe_features])

features_test = features_test.drop(ohe_features, axis=1)

In [27]:
# скелером, который обучен на ТРЕНИРОВОЧНОЙ ВЫБОРКЕ, масштабируем тестовую
features_test[num_features] = scaler.transform(
    features_test[num_features])

In [28]:
# смотрим на результат
features_test.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
7445,-1.369326,0.560665,-0.336987,-1.237830,-0.908179,1,0,-0.086537,0.0,0.0,0.0
8620,1.232367,0.090079,1.041028,-1.237830,0.809300,0,1,-0.537457,0.0,0.0,0.0
1714,0.840048,0.560665,0.352020,1.231363,-0.908179,0,0,1.070393,1.0,0.0,1.0
5441,1.056856,-0.945210,-1.370498,0.951231,-0.908179,1,0,-0.576279,0.0,0.0,1.0
9001,0.406433,-0.662858,0.352020,0.767800,-0.908179,1,1,0.662068,0.0,1.0,1.0


То же самое делаем с валидационной выборкой

In [29]:
# энкодером, который обучен на ТРЕНИРОВОЧНОЙ ВЫБОРКЕ, кодируем тестовую
features_valid[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(features_valid[ohe_features])

features_valid = features_valid.drop(ohe_features, axis=1)

In [30]:
# скелером, который обучен на ТРЕНИРОВОЧНОЙ ВЫБОРКЕ, масштабируем тестовую
features_valid[num_features] = scaler.transform(
    features_valid[num_features])

In [31]:
# смотрим на результат
features_valid.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
5170,1.707279,-0.756975,-0.336987,-1.237830,0.809300,1,1,0.718362,0.0,0.0,0.0
4180,-0.429826,-0.286389,1.730036,0.485538,0.809300,0,0,1.687305,1.0,0.0,1.0
7349,-0.171721,0.278313,0.352020,-0.269213,0.809300,0,1,0.824128,1.0,0.0,1.0
7469,0.385784,-0.380507,1.041028,0.464813,-0.908179,1,0,-1.118018,0.0,0.0,0.0
3467,-1.142194,0.278313,-1.370498,0.353837,0.809300,0,1,-0.107642,1.0,0.0,0.0


In [32]:
# получилось одинаковое количество признаков
features_train.shape, features_valid.shape, features_test.shape

((5454, 11), (1819, 11), (1818, 11))

In [34]:
# получились одинаковые признаки 
((features_train.columns != features_valid.columns) | 
 (features_train.columns != features_test.columns)).sum()

0

В данном разделе была проведена подготовка данных к работе с моделями. Типы данных в стоолбцах были правильными, пропусков не было, за исключением столбца "Tenure", где отсутствовало около 10 % данных. Строки с пропусками были убраны для обеспечения возможности обучения моделей. Также были проведены кодировка категориальных признаков методом OHE и масштабирование численных признаков методом стандартизации.

## Исследование задачи

Рассмотрим баланс классов

In [35]:
target_train.shape

(5454,)

In [36]:
target_train.sum()

1126

Т.к. пропусков нет, имеем, что значение целевого признака положительно примерно в 20% случаев

Обучим модель без учета дисбаланса классов. Используем модель логистической регрессии.

In [37]:
model = LogisticRegression(solver='liblinear', random_state=12345)

In [38]:
model.fit(features_train, target_train)

LogisticRegression(random_state=12345, solver='liblinear')

In [39]:
predictions = model.predict(features_valid) #Предсказания для тестовой выборки

Полнота прогноза составила 

In [40]:
recall_score(target_valid, predictions)

0.21082621082621084

Точность прогноза равна

In [41]:
precision_score(target_valid, predictions)

0.6434782608695652

F1-мера составила

In [42]:
f1_score(target_valid, predictions)

0.3175965665236052

Найдем значение ROC-AUC

In [43]:
probabilities_valid = model.predict_proba(features_valid)

In [44]:
probabilities_one_valid = probabilities_valid[:,1]

In [45]:
roc_auc_score(target_valid, probabilities_one_valid)

0.780939239386106

Получили значение ROC_AUC равное 0.78. При учете дисбаланса классов, оно, как и остальные метрики должно возрасти

Константная модель будет всегда предсказывать 1.

In [46]:
target_pred_constant = pd.Series([1 for x in range(0, features_valid.shape[0])])

In [47]:
recall_score(target_valid, target_pred_constant)

1.0

In [48]:
precision_score(target_valid, target_pred_constant)

0.19296316657504123

In [49]:
f1_score(target_valid, target_pred_constant)

0.3235023041474654

Полнота равна 1, константная модель предсказуемо нашла все TP. Точность отражает соотношение классов, значение f1-меры примерно равно значению f1-меры модели. Хоть константная модель и показала похожее значение f1-меры, ее использование было бы менее эффективным, поскольку пришлось бы рассматривать всех клиентов как находящихся в зоне риска.

Константная модель, всегда предсказывающая нули, имела бы все выбранные метрики равными нулю, поскольку числители формул для этих метрик обнуляются при отсутствии TP-ответов.

In [50]:
forest_model = RandomForestClassifier(random_state=12345)

Значения гиперпараметров для GridSearchCV

In [51]:
search_space = {
    "n_estimators" : [x for x in range(10, 110, 10)],
    "max_depth" : [x for x in range(1,11)]
}

Создаем GridSearchCV объект

In [52]:
GS = GridSearchCV(estimator = forest_model,
                  param_grid = search_space,
                  scoring = ["recall", "precision", "f1"], #sklearn.metrics.SCORERS.keys()
                  refit = "f1",
                  )

In [53]:
GS.fit(features_train, target_train)

GridSearchCV(estimator=RandomForestClassifier(random_state=12345),
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90,
                                          100]},
             refit='f1', scoring=['recall', 'precision', 'f1'])

In [54]:
best_forest = GS.best_estimator_ #модель случайного леса с оптимальными параметрами 
#"n_estimators" и "max_depth" по метрике "f1"

In [55]:
best_forest.fit(features_train, target_train)

RandomForestClassifier(max_depth=10, n_estimators=40, random_state=12345)

In [56]:
predictions = best_forest.predict(features_valid) #Предсказания для тестовой выборки

Полнота прогноза составила 

In [57]:
recall_score(target_valid, predictions)

0.43304843304843305

Точность прогноза равна

In [58]:
precision_score(target_valid, predictions)

0.8128342245989305

F1-мера составила

In [98]:
f1_score(target_valid, predictions)

0.5650557620817843

Найдем значение ROC-AUC

In [99]:
probabilities_valid = best_forest.predict_proba(features_valid)

In [100]:
probabilities_one_valid = probabilities_valid[:,1]

In [101]:
roc_auc_score(target_valid, probabilities_one_valid)

0.8602125495858466

Получили значение ROC_AUC равное 0.86. Эта метрика также показала прирост качества модели

In [157]:
tree_model = DecisionTreeClassifier(random_state=12345)

Создаем GridSearchCV объект

In [158]:
GS = GridSearchCV(estimator = tree_model,
                  param_grid = {'max_depth':[x for x in range(0,15)]},
                  scoring = ["recall", "precision", "f1"], #sklearn.metrics.SCORERS.keys()
                  refit = "f1",
                  )

In [159]:
GS.fit(features_train, target_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=12345),
             param_grid={'max_depth': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14]},
             refit='f1', scoring=['recall', 'precision', 'f1'])

In [160]:
best_tree = GS.best_estimator_ #модель случайного леса с оптимальными параметрами 
#"n_estimators" и "max_depth" по метрике "f1"

In [161]:
best_tree.fit(features_train, target_train)

DecisionTreeClassifier(max_depth=7, random_state=12345)

In [162]:
predictions = best_tree.predict(features_valid) #Предсказания для тестовой выборки

Полнота прогноза составила 

In [163]:
recall_score(target_valid, predictions)

0.46438746438746437

Точность прогноза равна

In [164]:
precision_score(target_valid, predictions)

0.6848739495798319

F1-мера составила

In [165]:
f1_score(target_valid, predictions)

0.5534804753820034

Найдем значение ROC-AUC

In [166]:
probabilities_valid = best_tree.predict_proba(features_valid)

In [168]:
probabilities_one_valid = probabilities_valid[:,1]

In [169]:
roc_auc_score(target_valid, probabilities_one_valid)

0.8280166825807154

Получили значение ROC_AUC равное 0.82

На данном этапе наилучшие результаты показала модель случайного леса (если сравнивать по f1-мере). Гиперпараметры были подобраны с помощью GridSearchCV. Наиболее эффективна модель сохранена в переменной best_forest

## Борьба с дисбалансом

<b> Учтем дисбаланс методом взвешивания классов </b>

**Логистическая регрессия**

In [124]:
model = LogisticRegression(solver='liblinear', random_state=12345, class_weight='balanced')

In [125]:
model.fit(features_train, target_train)

LogisticRegression(class_weight='balanced', random_state=12345,
                   solver='liblinear')

In [127]:
predictions = model.predict(features_valid) #Предсказания для тестовой выборки

Полнота прогноза составила 

In [128]:
recall_score(target_valid, predictions)

0.7037037037037037

Точность прогноза равна

In [130]:
precision_score(target_valid, predictions)

0.39082278481012656

F1-мера составила

In [131]:
f1_score(target_valid, predictions)

0.5025432349949135

Найдем значение ROC-AUC

In [137]:
probabilities_valid = model.predict_proba(features_valid)

In [138]:
probabilities_one_valid = probabilities_valid[:,1]

In [139]:
roc_auc_score(target_valid, probabilities_one_valid)

0.7817621121435836

Получили значение ROC_AUC осталось равным 0.78.

**Случайный лес**

In [140]:
forest_model = RandomForestClassifier(random_state=12345, class_weight='balanced')

Значения гиперпараметров для GridSearchCV

In [141]:
search_space = {
    "n_estimators" : [x for x in range(10, 110, 10)],
    "max_depth" : [x for x in range(1,11)]
}

Создаем GridSearchCV объект

In [142]:
GS = GridSearchCV(estimator = forest_model,
                  param_grid = search_space,
                  scoring = ["recall", "precision", "f1"], #sklearn.metrics.SCORERS.keys()
                  refit = "f1",
                  )

In [143]:
GS.fit(features_train, target_train)

GridSearchCV(estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=12345),
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90,
                                          100]},
             refit='f1', scoring=['recall', 'precision', 'f1'])

In [144]:
best_forest = GS.best_estimator_ #модель случайного леса с оптимальными параметрами 
#"n_estimators" и "max_depth" по метрике "f1"

In [145]:
best_forest.fit(features_train, target_train)

RandomForestClassifier(class_weight='balanced', max_depth=9, n_estimators=90,
                       random_state=12345)

In [146]:
predictions = best_forest.predict(features_valid) #Предсказания для тестовой выборки

Полнота прогноза составила 

In [147]:
recall_score(target_valid, predictions)

0.6353276353276354

Полнота выросла с 0.43 до 0.64

Точность прогноза равна

In [148]:
precision_score(target_valid, predictions)

0.5603015075376885

Точность упала с 0,81 до 0,56

F1-мера составила

In [149]:
f1_score(target_valid, predictions)

0.595460614152203

f1-мера выросла с 0,56 до 0,59

Найдем значение ROC-AUC

In [150]:
probabilities_valid = best_forest.predict_proba(features_valid)

In [151]:
probabilities_one_valid = probabilities_valid[:,1]

In [152]:
roc_auc_score(target_valid, probabilities_one_valid)

0.8571170730571276

ROC_AUC практически не изменилось

**Решающее дерево**

In [171]:
tree_model = DecisionTreeClassifier(random_state=12345, class_weight='balanced')

Создаем GridSearchCV объект

In [172]:
GS = GridSearchCV(estimator = tree_model,
                  param_grid = {'max_depth':[x for x in range(0,15)]},
                  scoring = ["recall", "precision", "f1"], #sklearn.metrics.SCORERS.keys()
                  refit = "f1",
                  )

In [173]:
GS.fit(features_train, target_train)

GridSearchCV(estimator=DecisionTreeClassifier(class_weight='balanced',
                                              random_state=12345),
             param_grid={'max_depth': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14]},
             refit='f1', scoring=['recall', 'precision', 'f1'])

In [174]:
best_tree = GS.best_estimator_ #модель случайного леса с оптимальными параметрами 
#"n_estimators" и "max_depth" по метрике "f1"

In [175]:
best_tree.fit(features_train, target_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=6, random_state=12345)

In [176]:
predictions = best_tree.predict(features_valid) #Предсказания для тестовой выборки

Полнота прогноза составила 

In [177]:
recall_score(target_valid, predictions)

0.7891737891737892

Полнота прогноза возрасла с 0,44 до 0,79

In [179]:
precision_score(target_valid, predictions)

0.4085545722713864

Точность упала с 0,68 до 0,41

F1-мера составила

In [180]:
f1_score(target_valid, predictions)

0.5383867832847424

Значение f1-меры упало с 0,56 до 0,54

Найдем значение ROC-AUC

In [183]:
probabilities_valid = best_tree.predict_proba(features_valid)

In [184]:
probabilities_one_valid = probabilities_valid[:,1]

In [185]:
roc_auc_score(target_valid, probabilities_one_valid)

0.8378513705489182

<b> Учтем дисбаланс методом upsampling </b>

Делаем upsampling

In [192]:
oversample = SMOTE(random_state=12345)

In [193]:
features_train_up, target_train_up = oversample.fit_resample(features_train, target_train)

Посмотрим, как работают модели после апсемлинга

**Логистическая регрессия**

In [194]:
model = LogisticRegression(solver='liblinear', random_state=12345)

In [195]:
model.fit(features_train_up, target_train_up)

LogisticRegression(random_state=12345, solver='liblinear')

In [196]:
predictions = model.predict(features_valid) #Предсказания для тестовой выборки

Полнота прогноза составила 

In [197]:
recall_score(target_valid, predictions)

0.6609686609686609

Точность прогноза равна

In [198]:
precision_score(target_valid, predictions)

0.37540453074433655

F1-мера составила

In [199]:
f1_score(target_valid, predictions)

0.47884416924664597

Найдем значение ROC-AUC

In [200]:
probabilities_valid = model.predict_proba(features_valid)

In [201]:
probabilities_one_valid = probabilities_valid[:,1]

In [202]:
roc_auc_score(target_valid, probabilities_one_valid)

0.7775021930335282

In [203]:
forest_model = RandomForestClassifier(random_state=12345)

Значения гиперпараметров для GridSearchCV

In [204]:
search_space = {
    "n_estimators" : [x for x in range(10, 110, 10)],
    "max_depth" : [x for x in range(1,11)]
}

Создаем GridSearchCV объект

In [205]:
GS = GridSearchCV(estimator = forest_model,
                  param_grid = search_space,
                  scoring = ["recall", "precision", "f1"], #sklearn.metrics.SCORERS.keys()
                  refit = "f1",
                  )

In [206]:
GS.fit(features_train_up, target_train_up)

GridSearchCV(estimator=RandomForestClassifier(random_state=12345),
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90,
                                          100]},
             refit='f1', scoring=['recall', 'precision', 'f1'])

In [207]:
best_forest = GS.best_estimator_ #модель случайного леса с оптимальными параметрами 
#"n_estimators" и "max_depth" по метрике "f1"

In [209]:
best_forest.fit(features_train_up, target_train_up)

RandomForestClassifier(max_depth=10, n_estimators=40, random_state=12345)

In [210]:
predictions = best_forest.predict(features_valid) #Предсказания для тестовой выборки

Полнота прогноза составила 

In [211]:
recall_score(target_valid, predictions)

0.6381766381766382

Точность прогноза равна

In [212]:
precision_score(target_valid, predictions)

0.5586034912718204

F1-мера составила

In [213]:
f1_score(target_valid, predictions)

0.5957446808510638

Найдем значение ROC-AUC

In [215]:
probabilities_valid = best_forest.predict_proba(features_valid)

In [216]:
probabilities_one_valid = probabilities_valid[:,1]

In [217]:
roc_auc_score(target_valid, probabilities_one_valid)

0.853886715262737

**Решающее дерево**

In [218]:
tree_model = DecisionTreeClassifier(random_state=12345)

Создаем GridSearchCV объект

In [219]:
GS = GridSearchCV(estimator = tree_model,
                  param_grid = {'max_depth':[x for x in range(0,15)]},
                  scoring = ["recall", "precision", "f1"], #sklearn.metrics.SCORERS.keys()
                  refit = "f1",
                  )

In [221]:
GS.fit(features_train_up, target_train_up)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=12345),
             param_grid={'max_depth': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14]},
             refit='f1', scoring=['recall', 'precision', 'f1'])

In [222]:
best_tree = GS.best_estimator_ #модель случайного леса с оптимальными параметрами 
#"n_estimators" и "max_depth" по метрике "f1"

In [223]:
best_tree.fit(features_train_up, target_train_up)

DecisionTreeClassifier(max_depth=14, random_state=12345)

In [225]:
predictions = best_tree.predict(features_valid) #Предсказания для тестовой выборки

Полнота прогноза составила 

In [226]:
recall_score(target_valid, predictions)

0.5527065527065527

Точность прогноза равна

In [229]:
precision_score(target_valid, predictions)

0.42543859649122806

F1-мера составила

In [230]:
f1_score(target_valid, predictions)

0.48079306071871125

Найдем значение ROC-AUC

In [231]:
probabilities_valid = best_tree.predict_proba(features_valid)

In [232]:
probabilities_one_valid = probabilities_valid[:,1]

In [233]:
roc_auc_score(target_valid, probabilities_one_valid)

0.6928093729864847

  
<p>&nbsp;&nbsp;&nbsp;&nbsp;В ходе исследования была выявлена закономерность: при учете дисбаланса классов увеличивается полнота прогнозирования, но снижается его точность, что приводит к не столь значительному изменению f1-меры, по отношению к изменениям полноты или точности.</p>
<p>&nbsp;&nbsp;&nbsp;&nbsp; Наилучшие результаты показала модель случайного леса, давшая при апсемплинге самое высокое значение f1 - 0,60. Проверим эту модель на тестовой выборке.</p>

## Тестирование модели

Для тестирования выбранной модели используем тестовую выборку

In [238]:
predictions_test = best_forest.predict(features_test)

In [244]:
recall_score(target_test, predictions_test)

0.6790450928381963

In [245]:
precision_score(target_test, predictions_test)

0.5967365967365967

In [239]:
f1_score(target_test, predictions_test)

0.6352357320099254

In [243]:
probabilities_test = best_forest.predict_proba(features_test)
probabilities_one_test = probabilities_test[:,1]
roc_auc_score(target_test, probabilities_one_test)

0.8681949795400703

С тестовой выборкой модель справилась. При этом значение f1-метрики равно 0.64, а значение ROC_AUC равно 0.87

In [246]:
best_forest

RandomForestClassifier(max_depth=10, n_estimators=40, random_state=12345)

<p>&nbsp;&nbsp;&nbsp;&nbsp;В ходе исследования были рассмотрены модели решающего дерева, логистической регрессии и случайного леса. При этом изначально в данных наблюдался дисбаланс классов, что снижало эффективноть моделей. Для решения этой проблемы были использованы два метода: метод взвешивания классов и метод 'upsampling'. Благодаря этому удалось добиться улучшения показателей.</p>      
<p>&nbsp;&nbsp;&nbsp;&nbsp;Наилучшие показатели дала модель случайного леса, обученная на выборке после апсемплинга. Гиперпараметры: max_depth=10, n_estimators=40, random_state=12345. f1 = 0,64, precision = 0,60, recall = 0,68.</p> 